# Orchestration

One sophisticated use of JSON mode is in breaking ambitious tasks into subtasks with JSON, then executing specific workflows for each subtask before fusing them together into a final result.

In this notebook, we'll explore how to use AI task orchestration to plan a 4-day trip to New York City. Breaking down complex tasks into smaller, manageable subtasks is a powerful technique that can help produce more detailed and coherent results.

## Setup

First, let's import the necessary libraries and set up our connection to the OpenAI API:

In [ ]:
from openai import OpenAI
import json

client = OpenAI()

def get_completion(prompt, system_prompt="You are a helpful assistant.", json_mode=False):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": prompt}],
        response_format={"type": "json_object"} if json_mode else None
    )
    return response.choices[0].message.content

## Checkpoint 1/3: Task Planning

To illustrate this point, let's create an agentic workflow for travel planning. Relying on the model's innate knowledge, we'll have it plan out a trip to NYC for a family of four on a budget.

First, we need to break down our complex task into smaller, manageable subtasks. We'll create a function that uses an LLM to analyze the task and return a JSON list of subtasks. To make things more interesting, we'll specify that each list item should be a dict with `text` and `budget` fields.

That way, the initial task will parcel out our total budget to subtasks, giving the subtask prompt useful context.

In [ ]:
initial_output = [{
    "text": "Text describing the subtask",
    "budget": "amount of money out of the total travel budget to spend on this task",
}
# and more...
]

Let's implement a simple task planning function. In the `<specified_format>` section, add an example of the JSON we should see returned. In the `<budget>` section of the prompt, specify a family budget of $5,000. Then, obtain the model output and store it in `json_string`. Load the JSON into Python in the following line, and then assign to `subtasks` the `"data"` field of the result.

In [ ]:
def create_subtask_list(prompt):
    return get_completion(prompt, json_mode=True)

prompt = """<task>Your task is to help plan a four day vacation to New York City for a family of four (we live in California.)
You'll do so by providing a list of subtasks involved with the trip.
Try to cover everything we'll need to have great family memories.
Output should be a JSON object with the key 'data' containing a list of subtasks in the format specified below.
Each content field should be a description of everything we need to figure out to accomplish this subtask on our trip.
Make the content field about one paragraph long.
</task>

<specified_format>
{"data": [
{"content": "Subtask description and instructions here",
"budget": "Portion of the budget that can be allocated to this subtask"
},
# and so on
]}}
</specified_format>

<budget>$5,000</budget>

Output ONLY valid JSON below. No markdown backticks.
"""

json_string = create_subtask_list(prompt)
result = json.loads(json_string)
subtasks = result["data"]
print(subtasks)

## Checkpoint 2/3: Subtask Execution

Now that we have our subtasks, we need to execute each one. We'll create a function that takes the subtasks from Checkpoint 1 and generates a response for each subtask using tailored prompts.

While we're deliberately keeping this very simple, imagine each of these subtasks involving their own sophisticated workflows—by breaking down our more complex task into these sub-units, we can accomplish a great deal.

Initialize an empty list and assign it to `subtask_solutions`. Pass the content and budget of the subtask to the designated fields in the prompt, and then add the model's result to the `subtask_solutions` list.

Then, in the `for` loop, call this function on each item in the subtask list.

In [ ]:
subtask_solutions = []

def execute_subtask(subtask):
    prompt = f"""<task>Your task is to help plan the specified aspect of a family of four's four day trip to New York City.
We're coming from California.
The task below is one aspect of our overall trip that we want you to plan out for us.
Output a 2-3 short sentences using your innate knowledge to offer your final recommendation on what to do and purchase.
We've attached a budget for this domain of our trip, out of a total budget of $5,000.
Itemize what you'd spend this subsection of the budget on to the best of your knowledge.
Being your output with a 1-3 word description of your domain, then the 2-3 short sentences and the itemized spending plan (if needed).
</task>

<subtask_content>
{subtask['content']}
</subtask_content>

<subtask_budget>
{subtask['budget']}
</subtask_budget>"""
    result = get_completion(prompt)
    subtask_solutions.append(result)

for subtask in subtasks:
    execute_subtask(subtask)

print(subtask_solutions)

## Checkpoint 3/3: Result Synthesis

Now we'll fuse all this verbose output into a single summary with only the most useful information.

We'll begin by concatenating the results of the subtask solutions into a single string.

Then specify the kind of report you'd like to get back from the fusion prompt. Put the details of what you want in your report under the "Output the following" line in the prompt. Some items to consider:

- Offer a detailed, time-based itinerary
- List out all purchases
- Specify locations, times, and contingency plans
- Describe your desired formatting (bullet points, numbered list, markdown headings?)

Finally, call the `fuse_solutions` function and assign it to `final_plan`.

In [ ]:
def fuse_solutions(subtask_solutions):
    # Concatenate all subtask solutions with carriage returns between them
    combined_solutions = "\n\n".join(subtask_solutions)
    
    prompt = f"""<task>Your task is to condense a detailed list of plans for our upcoming trip to NYC into a single, short summary.
Include only actionable information we can directly use to carry out our vacation.
Output the following:
- Offer a detailed, time-based itinerary
- List out all purchases
- Specify locations, times, and contingency plans
- Use bullet point lists
</task>

<detailed_plans>
{combined_solutions}
</detailed_plans>"""
    
    return get_completion(prompt)

final_plan = fuse_solutions(subtask_solutions)
print(final_plan)